In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from gensim import corpora, models, similarities

# Linear Models 
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB

from tqdm import tqdm

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Flatten, LSTM, Input
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping



C:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

14.348666667938232


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

idxList = []
for i in allTextToxicTrain.keys():
    #allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]
    idxList = np.unique(np.append(idxList, allTextToxicTrain[i]))
allTextToxicTrain = [str(train['cleanWordsAsText'][j]) for j in idxList]

In [8]:
columns = ["idExp","numFeatures", "algorithm", "Nfolds", "accuaracy", "logloss", "fmeasure"]
dfTestResults = pd.DataFrame(columns=columns)


In [9]:
listTopNFeatures = [10,50,100,200,500,1000]

In [10]:
# Test clasification
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()


,Unnamed: 0,id,comment_text,listOfCleanWords,cleanWordsAsText
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev...",yo bitch ja rule succesful ever whats hating s...
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']",rfc title fine imo
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['source', 'zawe', 'ashton', 'lapland']",source zawe ashton lapland
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd...",look back source information updated correct f...
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'article']",anonymously edit article


In [11]:
# PREDICCION MULTICLASS TEST
topNFeatures = listTopNFeatures[]
#Neutral data

tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxicTrain)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]

#Toxic data
selectedWordsTrainToxic = []
selectedWordsTrain = []

tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextToxicTrain)
featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]
selectedWordsTrainToxic = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]

# Get words
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        words = [selectedWordsTrainToxic[x]] + [selectedWordsTrainNoToxic[x]]
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique(words)))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]

allTrainText = [txt if txt is not np.nan else '' for txt in train['cleanWordsAsText']]
allTestText = [txt if txt is not np.nan else '' for txt in test['cleanWordsAsText']]
X_train = allTrainText
X_test = allTestText
yBinary = MultiLabelBinarizer().fit_transform(y)
y_train = yBinary


vectTrain = CountVectorizer(vocabulary=selectedWordsTrain)
X_train_bow = vectTrain.fit_transform(X_train).toarray()
X_test_bow = vectTrain.fit_transform(X_test).toarray()


In [12]:
# Word 2 Vec mean train list
modelWord2VecTrain = models.Word2Vec(train['listOfCleanWords'].tolist(), min_count=1)
#say_vector = modelWord2VecTrain['explanation']  # get vector for word
#say_vector

In [13]:
vectorizedTrainW2V = np.empty(len(train['listOfCleanWords']), dtype=list)
for i, lstOfCleanWords in enumerate(train['listOfCleanWords']): 
    size = len(lstOfCleanWords)
    if size > 0:
        vector = np.zeros(len(modelWord2VecTrain[lstOfCleanWords[0]]))
        for x in range(size):
            vector += modelWord2VecTrain[lstOfCleanWords[x]]
        vectorizedTrainW2V[i] = vector / size

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [14]:
#vectorizedTrainW2V

Using TensorFlow backend.


In [16]:
#CNN architecture
numClases = 7

#training params
batch_size = 512 
num_epochs = 8 

#model parameters
num_filters = 128 
weight_decay = 1e-4
outputDim = 100

In [17]:


print("training CNN ...")
model = Sequential()
model.add(Embedding(input_dim=len(X_train), output_dim=outputDim))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(numClases, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

training CNN ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         47871300  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          134464    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080

In [18]:
#model training
hist = model.fit(X_train_bow, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.1, shuffle=True, verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/8
 - 2030s - loss: 0.1867 - acc: 0.9501 - val_loss: 0.1772 - val_acc: 0.9534
Epoch 2/8
 - 2019s - loss: 0.1708 - acc: 0.9541 - val_loss: 0.1726 - val_acc: 0.9534
Epoch 3/8
 - 2022s - loss: 0.1685 - acc: 0.9541 - val_loss: 0.1722 - val_acc: 0.9534
Epoch 4/8
 - 2027s - loss: 0.1673 - acc: 0.9541 - val_loss: 0.1784 - val_acc: 0.9534
Epoch 5/8
 - 2026s - loss: 0.1662 - acc: 0.9541 - val_loss: 0.1698 - val_acc: 0.9534
Epoch 6/8
 - 2020s - loss: 0.1654 - acc: 0.9541 - val_loss: 0.1721 - val_acc: 0.9534
Epoch 7/8
 - 2039s - loss: 0.1649 - acc: 0.9541 - val_loss: 0.1698 - val_acc: 0.9534
Epoch 8/8
 - 2028s - loss: 0.1644 - acc: 0.9541 - val_loss: 0.1664 - val_acc: 0.9534


In [19]:
y_test_pred = model.predict(X_test_bow)

In [20]:
predicted = y_test_pred
columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
dfTestPredicted = pd.DataFrame(columns=columns)
for x in tqdm(range(len(test))):
    dfTestPredicted.loc[x] = [test['id'][x], predicted[x][1], predicted[x][2], predicted[x][3], predicted[x][4], predicted[x][5], predicted[x][6]]
dfTestPredicted.to_csv('../reports/testPred/predTestCNN'+ str(topNFeatures) +'.csv',encoding='utf-8', index=False)

  4%|▍         | 6584/153164 [00:09<03:41, 662.66it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 153164/153164 [22:45<00:00, 112.20it/s]


In [21]:
# LSTM
model = Sequential()
model.add(Embedding(len(X_train), output_dim=256))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(7, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])




In [24]:
model.fit(X_train_bow, y_train, batch_size=batch_size, epochs=min(num_epochs,2), validation_split=0.1);

Train on 143613 samples, validate on 15958 samples
Epoch 1/2


 49408/143613 [=========>....................] - ETA: 1:50:32 - loss: 0.2085 - acc: 0.94 - ETA: 1:56:06 - loss: 0.2003 - acc: 0.94 - ETA: 1:58:55 - loss: 0.1882 - acc: 0.94 - ETA: 1:57:48 - loss: 0.1782 - acc: 0.95 - ETA: 1:56:22 - loss: 0.1773 - acc: 0.95 - ETA: 1:56:43 - loss: 0.1755 - acc: 0.95 - ETA: 2:01:03 - loss: 0.1735 - acc: 0.95 - ETA: 2:00:44 - loss: 0.1741 - acc: 0.95 - ETA: 2:00:18 - loss: 0.1702 - acc: 0.95 - ETA: 1:59:51 - loss: 0.1669 - acc: 0.95 - ETA: 1:59:26 - loss: 0.1695 - acc: 0.95 - ETA: 1:59:15 - loss: 0.1683 - acc: 0.95 - ETA: 1:59:01 - loss: 0.1676 - acc: 0.95 - ETA: 1:58:47 - loss: 0.1711 - acc: 0.95 - ETA: 1:58:27 - loss: 0.1722 - acc: 0.95 - ETA: 1:58:11 - loss: 0.1747 - acc: 0.95 - ETA: 1:57:51 - loss: 0.1741 - acc: 0.95 - ETA: 1:57:24 - loss: 0.1729 - acc: 0.95 - ETA: 1:57:16 - loss: 0.1716 - acc: 0.95 - ETA: 1:57:07 - loss: 0.1713 - acc: 0.95 - ETA: 1:56:52 - loss: 0.1712 - acc: 0.95 - ETA: 1:56:40 - loss: 0.1696 - acc: 0.95 - ETA: 1:56:30 - loss: 0.1703

100864/143613 [====================>.........] - ETA: 1:13:43 - loss: 0.1688 - acc: 0.95 - ETA: 1:13:31 - loss: 0.1685 - acc: 0.95 - ETA: 1:13:18 - loss: 0.1685 - acc: 0.95 - ETA: 1:13:06 - loss: 0.1685 - acc: 0.95 - ETA: 1:12:54 - loss: 0.1684 - acc: 0.95 - ETA: 1:12:41 - loss: 0.1685 - acc: 0.95 - ETA: 1:12:29 - loss: 0.1685 - acc: 0.95 - ETA: 1:12:16 - loss: 0.1685 - acc: 0.95 - ETA: 1:12:03 - loss: 0.1684 - acc: 0.95 - ETA: 1:11:50 - loss: 0.1682 - acc: 0.95 - ETA: 1:11:38 - loss: 0.1682 - acc: 0.95 - ETA: 1:11:26 - loss: 0.1683 - acc: 0.95 - ETA: 1:11:13 - loss: 0.1683 - acc: 0.95 - ETA: 1:11:01 - loss: 0.1684 - acc: 0.95 - ETA: 1:10:48 - loss: 0.1684 - acc: 0.95 - ETA: 1:10:36 - loss: 0.1685 - acc: 0.95 - ETA: 1:10:23 - loss: 0.1685 - acc: 0.95 - ETA: 1:10:11 - loss: 0.1686 - acc: 0.95 - ETA: 1:09:59 - loss: 0.1684 - acc: 0.95 - ETA: 1:09:46 - loss: 0.1683 - acc: 0.95 - ETA: 1:09:34 - loss: 0.1684 - acc: 0.95 - ETA: 1:09:22 - loss: 0.1684 - acc: 0.95 - ETA: 1:09:09 - loss: 0.1684

143613/143613 [==============================] - ETA: 33:16 - loss: 0.1692 - acc: 0.95 - ETA: 33:04 - loss: 0.1691 - acc: 0.95 - ETA: 32:52 - loss: 0.1692 - acc: 0.95 - ETA: 32:40 - loss: 0.1692 - acc: 0.95 - ETA: 32:28 - loss: 0.1692 - acc: 0.95 - ETA: 32:16 - loss: 0.1691 - acc: 0.95 - ETA: 32:04 - loss: 0.1691 - acc: 0.95 - ETA: 31:52 - loss: 0.1690 - acc: 0.95 - ETA: 31:40 - loss: 0.1691 - acc: 0.95 - ETA: 31:28 - loss: 0.1692 - acc: 0.95 - ETA: 31:16 - loss: 0.1692 - acc: 0.95 - ETA: 31:04 - loss: 0.1692 - acc: 0.95 - ETA: 30:52 - loss: 0.1692 - acc: 0.95 - ETA: 30:40 - loss: 0.1693 - acc: 0.95 - ETA: 30:28 - loss: 0.1693 - acc: 0.95 - ETA: 30:16 - loss: 0.1692 - acc: 0.95 - ETA: 30:03 - loss: 0.1692 - acc: 0.95 - ETA: 29:51 - loss: 0.1691 - acc: 0.95 - ETA: 29:39 - loss: 0.1690 - acc: 0.95 - ETA: 29:27 - loss: 0.1689 - acc: 0.95 - ETA: 29:15 - loss: 0.1689 - acc: 0.95 - ETA: 29:03 - loss: 0.1688 - acc: 0.95 - ETA: 28:51 - loss: 0.1688 - acc: 0.95 - ETA: 28:39 - loss: 0.1688 - acc

 49664/143613 [=========>....................] - ETA: 1:54:57 - loss: 0.1471 - acc: 0.96 - ETA: 1:57:03 - loss: 0.1546 - acc: 0.95 - ETA: 1:56:38 - loss: 0.1597 - acc: 0.95 - ETA: 1:56:15 - loss: 0.1638 - acc: 0.95 - ETA: 1:55:26 - loss: 0.1634 - acc: 0.95 - ETA: 1:54:19 - loss: 0.1643 - acc: 0.95 - ETA: 1:53:31 - loss: 0.1661 - acc: 0.95 - ETA: 1:52:48 - loss: 0.1653 - acc: 0.95 - ETA: 1:52:08 - loss: 0.1701 - acc: 0.95 - ETA: 1:51:40 - loss: 0.1706 - acc: 0.95 - ETA: 1:51:21 - loss: 0.1698 - acc: 0.95 - ETA: 1:50:52 - loss: 0.1709 - acc: 0.95 - ETA: 1:50:29 - loss: 0.1719 - acc: 0.95 - ETA: 1:50:06 - loss: 0.1710 - acc: 0.95 - ETA: 1:49:54 - loss: 0.1720 - acc: 0.95 - ETA: 1:49:35 - loss: 0.1703 - acc: 0.95 - ETA: 1:49:24 - loss: 0.1681 - acc: 0.95 - ETA: 1:49:03 - loss: 0.1703 - acc: 0.95 - ETA: 1:48:52 - loss: 0.1716 - acc: 0.95 - ETA: 1:48:48 - loss: 0.1716 - acc: 0.95 - ETA: 1:48:36 - loss: 0.1706 - acc: 0.95 - ETA: 1:48:30 - loss: 0.1696 - acc: 0.95 - ETA: 1:48:24 - loss: 0.1692

101120/143613 [====================>.........] - ETA: 1:13:00 - loss: 0.1695 - acc: 0.95 - ETA: 1:12:48 - loss: 0.1696 - acc: 0.95 - ETA: 1:12:36 - loss: 0.1697 - acc: 0.95 - ETA: 1:12:24 - loss: 0.1698 - acc: 0.95 - ETA: 1:12:11 - loss: 0.1700 - acc: 0.95 - ETA: 1:11:59 - loss: 0.1700 - acc: 0.95 - ETA: 1:11:46 - loss: 0.1700 - acc: 0.95 - ETA: 1:11:34 - loss: 0.1700 - acc: 0.95 - ETA: 1:11:23 - loss: 0.1701 - acc: 0.95 - ETA: 1:11:14 - loss: 0.1702 - acc: 0.95 - ETA: 1:11:04 - loss: 0.1699 - acc: 0.95 - ETA: 1:10:53 - loss: 0.1697 - acc: 0.95 - ETA: 1:10:42 - loss: 0.1697 - acc: 0.95 - ETA: 1:10:30 - loss: 0.1696 - acc: 0.95 - ETA: 1:10:19 - loss: 0.1699 - acc: 0.95 - ETA: 1:10:08 - loss: 0.1700 - acc: 0.95 - ETA: 1:09:56 - loss: 0.1699 - acc: 0.95 - ETA: 1:09:44 - loss: 0.1698 - acc: 0.95 - ETA: 1:09:33 - loss: 0.1699 - acc: 0.95 - ETA: 1:09:21 - loss: 0.1698 - acc: 0.95 - ETA: 1:09:10 - loss: 0.1697 - acc: 0.95 - ETA: 1:08:59 - loss: 0.1697 - acc: 0.95 - ETA: 1:08:48 - loss: 0.1697

143613/143613 [==============================] - ETA: 34:12 - loss: 0.1693 - acc: 0.95 - ETA: 34:01 - loss: 0.1693 - acc: 0.95 - ETA: 33:49 - loss: 0.1693 - acc: 0.95 - ETA: 33:38 - loss: 0.1692 - acc: 0.95 - ETA: 33:26 - loss: 0.1693 - acc: 0.95 - ETA: 33:15 - loss: 0.1692 - acc: 0.95 - ETA: 33:04 - loss: 0.1691 - acc: 0.95 - ETA: 32:52 - loss: 0.1691 - acc: 0.95 - ETA: 32:41 - loss: 0.1690 - acc: 0.95 - ETA: 32:29 - loss: 0.1691 - acc: 0.95 - ETA: 32:17 - loss: 0.1691 - acc: 0.95 - ETA: 32:06 - loss: 0.1691 - acc: 0.95 - ETA: 31:54 - loss: 0.1690 - acc: 0.95 - ETA: 31:43 - loss: 0.1690 - acc: 0.95 - ETA: 31:31 - loss: 0.1690 - acc: 0.95 - ETA: 31:19 - loss: 0.1690 - acc: 0.95 - ETA: 31:08 - loss: 0.1691 - acc: 0.95 - ETA: 30:56 - loss: 0.1690 - acc: 0.95 - ETA: 30:45 - loss: 0.1688 - acc: 0.95 - ETA: 30:33 - loss: 0.1688 - acc: 0.95 - ETA: 30:21 - loss: 0.1688 - acc: 0.95 - ETA: 30:10 - loss: 0.1688 - acc: 0.95 - ETA: 29:58 - loss: 0.1688 - acc: 0.95 - ETA: 29:46 - loss: 0.1689 - acc

In [25]:
predicted = model.predict(X_test_bow, batch_size=1024, verbose=1)
columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
dfTestPredicted = pd.DataFrame(columns=columns)
for x in tqdm(range(len(test))):
    dfTestPredicted.loc[x] = [test['id'][x], predicted[x][1], predicted[x][2], predicted[x][3], predicted[x][4], predicted[x][5], predicted[x][6]]
dfTestPredicted.to_csv('../reports/testPred/predTestLSTM'+ str(topNFeatures) +'.csv',encoding='utf-8', index=False)

153164/153164 [==============================] - ETA: 1:01:2 - ETA: 1:01:1 - ETA: 1:00:4 - ETA: 58:43  - ETA: 58:0 - ETA: 57:5 - ETA: 57:4 - ETA: 57:3 - ETA: 57:1 - ETA: 57:0 - ETA: 56:4 - ETA: 56:2 - ETA: 55:5 - ETA: 55:3 - ETA: 55:1 - ETA: 54:5 - ETA: 54:2 - ETA: 54:0 - ETA: 53:4 - ETA: 52:5 - ETA: 52:1 - ETA: 51:2 - ETA: 50:4 - ETA: 50:0 - ETA: 49:2 - ETA: 48:4 - ETA: 48:0 - ETA: 47:2 - ETA: 46:5 - ETA: 46:1 - ETA: 45:3 - ETA: 44:5 - ETA: 44:2 - ETA: 43:5 - ETA: 43:3 - ETA: 43:1 - ETA: 43:0 - ETA: 42:4 - ETA: 42:2 - ETA: 42:0 - ETA: 41:4 - ETA: 41:2 - ETA: 41:0 - ETA: 40:4 - ETA: 40:2 - ETA: 39:5 - ETA: 39:3 - ETA: 39:1 - ETA: 38:5 - ETA: 38:3 - ETA: 38:1 - ETA: 37:5 - ETA: 37:3 - ETA: 37:0 - ETA: 36:4 - ETA: 36:2 - ETA: 36:0 - ETA: 35:4 - ETA: 35:1 - ETA: 34:5 - ETA: 34:3 - ETA: 34:1 - ETA: 33:5 - ETA: 33:2 - ETA: 33:0 - ETA: 32:4 - ETA: 32:2 - ETA: 31:5 - ETA: 31:3 - ETA: 31:1 - ETA: 30:5 - ETA: 30:2 - ETA: 30:0 - ETA: 29:4 - ETA: 29:1 - ETA: 28:5 - ETA: 28:3 - ETA: 28:0 - ETA: 27

  2%|▏         | 3493/153164 [00:09<07:05, 352.12it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 153164/153164 [52:13<00:00, 48.87it/s]


In [ ]:
##################################################################


In [26]:
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 175303 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 200 # max number of words in a comment to use

#training params
batch_size = 256 
num_epochs = 2 

In [27]:
all_training_words = [word for tokens in train["listOfCleanWords"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in train["listOfCleanWords"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

5534218 words total, with a vocabulary size of 158652
Max sentence length is 1250


In [28]:
all_test_words = [word for tokens in test["listOfCleanWords"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in test["listOfCleanWords"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

48908922 words total, with a vocabulary size of 31
Max sentence length is 10001


In [51]:
from keras.preprocessing.sequence import pad_sequences
import gensim
from keras.layers import Dense, Input, Flatten, Dropout, Merge

label_names = ["notoxic", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]


In [30]:


word2vec_path = "../data/raw/GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)



In [33]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['listOfCleanWords'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [34]:
training_embeddings = get_word2vec_embeddings(word2vec, train, generate_missing=True)

In [38]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(train["comment_text"].tolist())
training_sequences = tokenizer.texts_to_sequences(train["comment_text"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

Found 210337 unique tokens.
(210338, 300)


In [64]:
train_embedding_weights[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [40]:
test_sequences = tokenizer.texts_to_sequences(train["comment_text"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [41]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Merge(mode='concat', concat_axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model


In [42]:
x_train = train_cnn_data
y_tr = y_train

In [52]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)), False)

C:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 200, 300)     63101400    input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 198, 128)     115328      embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_11 (Conv1D)              (None, 197, 128)     153728      embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_12 

In [53]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [54]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2


 51968/143613 [=========>....................] - ETA: 23:00 - loss: 0.6693 - acc: 0.58 - ETA: 19:41 - loss: 0.5040 - acc: 0.74 - ETA: 18:25 - loss: 0.4002 - acc: 0.81 - ETA: 17:38 - loss: 0.3757 - acc: 0.85 - ETA: 17:14 - loss: 0.3640 - acc: 0.86 - ETA: 17:01 - loss: 0.3571 - acc: 0.88 - ETA: 16:46 - loss: 0.3426 - acc: 0.89 - ETA: 16:44 - loss: 0.3308 - acc: 0.90 - ETA: 16:44 - loss: 0.3140 - acc: 0.90 - ETA: 16:33 - loss: 0.3069 - acc: 0.91 - ETA: 16:24 - loss: 0.2989 - acc: 0.91 - ETA: 16:22 - loss: 0.2946 - acc: 0.91 - ETA: 16:17 - loss: 0.2905 - acc: 0.91 - ETA: 16:12 - loss: 0.2866 - acc: 0.92 - ETA: 16:06 - loss: 0.2820 - acc: 0.92 - ETA: 16:01 - loss: 0.2795 - acc: 0.92 - ETA: 15:58 - loss: 0.2788 - acc: 0.92 - ETA: 15:53 - loss: 0.2731 - acc: 0.92 - ETA: 15:51 - loss: 0.2679 - acc: 0.92 - ETA: 15:46 - loss: 0.2646 - acc: 0.92 - ETA: 15:41 - loss: 0.2603 - acc: 0.93 - ETA: 15:36 - loss: 0.2549 - acc: 0.93 - ETA: 15:33 - loss: 0.2521 - acc: 0.93 - ETA: 15:31 - loss: 0.2488 - acc

104192/143613 [====================>.........] - ETA: 9:48 - loss: 0.1521 - acc: 0.951 - ETA: 9:47 - loss: 0.1520 - acc: 0.951 - ETA: 9:45 - loss: 0.1519 - acc: 0.951 - ETA: 9:44 - loss: 0.1516 - acc: 0.951 - ETA: 9:42 - loss: 0.1515 - acc: 0.951 - ETA: 9:41 - loss: 0.1516 - acc: 0.951 - ETA: 9:39 - loss: 0.1515 - acc: 0.951 - ETA: 9:37 - loss: 0.1514 - acc: 0.951 - ETA: 9:36 - loss: 0.1512 - acc: 0.951 - ETA: 9:34 - loss: 0.1511 - acc: 0.951 - ETA: 9:32 - loss: 0.1509 - acc: 0.951 - ETA: 9:31 - loss: 0.1507 - acc: 0.951 - ETA: 9:29 - loss: 0.1504 - acc: 0.951 - ETA: 9:27 - loss: 0.1503 - acc: 0.951 - ETA: 9:26 - loss: 0.1502 - acc: 0.951 - ETA: 9:24 - loss: 0.1498 - acc: 0.951 - ETA: 9:22 - loss: 0.1496 - acc: 0.951 - ETA: 9:21 - loss: 0.1494 - acc: 0.951 - ETA: 9:19 - loss: 0.1493 - acc: 0.951 - ETA: 9:17 - loss: 0.1491 - acc: 0.951 - ETA: 9:16 - loss: 0.1491 - acc: 0.951 - ETA: 9:14 - loss: 0.1487 - acc: 0.951 - ETA: 9:12 - loss: 0.1487 - acc: 0.951 - ETA: 9:11 - loss: 0.1486 - acc:

143613/143613 [==============================] - ETA: 4:13 - loss: 0.1320 - acc: 0.955 - ETA: 4:11 - loss: 0.1320 - acc: 0.955 - ETA: 4:10 - loss: 0.1320 - acc: 0.955 - ETA: 4:08 - loss: 0.1319 - acc: 0.955 - ETA: 4:06 - loss: 0.1318 - acc: 0.955 - ETA: 4:05 - loss: 0.1317 - acc: 0.955 - ETA: 4:03 - loss: 0.1317 - acc: 0.955 - ETA: 4:01 - loss: 0.1316 - acc: 0.955 - ETA: 4:00 - loss: 0.1315 - acc: 0.955 - ETA: 3:58 - loss: 0.1314 - acc: 0.955 - ETA: 3:56 - loss: 0.1313 - acc: 0.955 - ETA: 3:55 - loss: 0.1313 - acc: 0.955 - ETA: 3:53 - loss: 0.1312 - acc: 0.955 - ETA: 3:51 - loss: 0.1311 - acc: 0.955 - ETA: 3:50 - loss: 0.1310 - acc: 0.955 - ETA: 3:48 - loss: 0.1310 - acc: 0.955 - ETA: 3:46 - loss: 0.1309 - acc: 0.955 - ETA: 3:45 - loss: 0.1308 - acc: 0.955 - ETA: 3:43 - loss: 0.1307 - acc: 0.955 - ETA: 3:41 - loss: 0.1307 - acc: 0.955 - ETA: 3:40 - loss: 0.1306 - acc: 0.955 - ETA: 3:38 - loss: 0.1305 - acc: 0.955 - ETA: 3:36 - loss: 0.1304 - acc: 0.955 - ETA: 3:35 - loss: 0.1304 - acc:

 52224/143613 [=========>....................] - ETA: 15:14 - loss: 0.0682 - acc: 0.97 - ETA: 15:14 - loss: 0.0832 - acc: 0.97 - ETA: 15:12 - loss: 0.0862 - acc: 0.96 - ETA: 15:13 - loss: 0.0891 - acc: 0.96 - ETA: 15:09 - loss: 0.0973 - acc: 0.96 - ETA: 15:08 - loss: 0.0961 - acc: 0.96 - ETA: 15:02 - loss: 0.0979 - acc: 0.96 - ETA: 14:59 - loss: 0.0980 - acc: 0.96 - ETA: 14:58 - loss: 0.0975 - acc: 0.96 - ETA: 14:55 - loss: 0.0992 - acc: 0.96 - ETA: 14:51 - loss: 0.0962 - acc: 0.96 - ETA: 14:51 - loss: 0.0977 - acc: 0.96 - ETA: 14:52 - loss: 0.0963 - acc: 0.96 - ETA: 14:48 - loss: 0.0972 - acc: 0.96 - ETA: 14:45 - loss: 0.0984 - acc: 0.96 - ETA: 14:42 - loss: 0.0976 - acc: 0.96 - ETA: 14:40 - loss: 0.0973 - acc: 0.96 - ETA: 14:38 - loss: 0.0982 - acc: 0.96 - ETA: 14:36 - loss: 0.0971 - acc: 0.96 - ETA: 14:35 - loss: 0.0960 - acc: 0.96 - ETA: 14:35 - loss: 0.0962 - acc: 0.96 - ETA: 14:33 - loss: 0.0984 - acc: 0.96 - ETA: 14:30 - loss: 0.0987 - acc: 0.96 - ETA: 14:28 - loss: 0.0983 - acc

104448/143613 [====================>.........] - ETA: 9:47 - loss: 0.0913 - acc: 0.967 - ETA: 9:45 - loss: 0.0913 - acc: 0.967 - ETA: 9:43 - loss: 0.0912 - acc: 0.967 - ETA: 9:42 - loss: 0.0912 - acc: 0.967 - ETA: 9:40 - loss: 0.0913 - acc: 0.967 - ETA: 9:39 - loss: 0.0914 - acc: 0.967 - ETA: 9:37 - loss: 0.0914 - acc: 0.967 - ETA: 9:35 - loss: 0.0914 - acc: 0.967 - ETA: 9:34 - loss: 0.0914 - acc: 0.967 - ETA: 9:32 - loss: 0.0916 - acc: 0.967 - ETA: 9:30 - loss: 0.0916 - acc: 0.967 - ETA: 9:29 - loss: 0.0916 - acc: 0.967 - ETA: 9:27 - loss: 0.0915 - acc: 0.967 - ETA: 9:25 - loss: 0.0914 - acc: 0.967 - ETA: 9:24 - loss: 0.0914 - acc: 0.967 - ETA: 9:22 - loss: 0.0914 - acc: 0.967 - ETA: 9:20 - loss: 0.0912 - acc: 0.967 - ETA: 9:19 - loss: 0.0912 - acc: 0.967 - ETA: 9:17 - loss: 0.0911 - acc: 0.967 - ETA: 9:16 - loss: 0.0911 - acc: 0.967 - ETA: 9:14 - loss: 0.0911 - acc: 0.967 - ETA: 9:12 - loss: 0.0910 - acc: 0.967 - ETA: 9:11 - loss: 0.0910 - acc: 0.967 - ETA: 9:09 - loss: 0.0909 - acc:

143613/143613 [==============================] - ETA: 4:15 - loss: 0.0878 - acc: 0.968 - ETA: 4:14 - loss: 0.0878 - acc: 0.968 - ETA: 4:12 - loss: 0.0878 - acc: 0.968 - ETA: 4:10 - loss: 0.0878 - acc: 0.968 - ETA: 4:09 - loss: 0.0878 - acc: 0.968 - ETA: 4:07 - loss: 0.0877 - acc: 0.968 - ETA: 4:05 - loss: 0.0878 - acc: 0.968 - ETA: 4:04 - loss: 0.0878 - acc: 0.968 - ETA: 4:02 - loss: 0.0878 - acc: 0.968 - ETA: 4:00 - loss: 0.0878 - acc: 0.968 - ETA: 3:59 - loss: 0.0878 - acc: 0.968 - ETA: 3:57 - loss: 0.0878 - acc: 0.968 - ETA: 3:55 - loss: 0.0877 - acc: 0.968 - ETA: 3:54 - loss: 0.0877 - acc: 0.968 - ETA: 3:52 - loss: 0.0877 - acc: 0.968 - ETA: 3:50 - loss: 0.0877 - acc: 0.968 - ETA: 3:49 - loss: 0.0876 - acc: 0.968 - ETA: 3:47 - loss: 0.0876 - acc: 0.968 - ETA: 3:45 - loss: 0.0876 - acc: 0.968 - ETA: 3:44 - loss: 0.0876 - acc: 0.968 - ETA: 3:42 - loss: 0.0876 - acc: 0.968 - ETA: 3:40 - loss: 0.0877 - acc: 0.968 - ETA: 3:39 - loss: 0.0876 - acc: 0.968 - ETA: 3:37 - loss: 0.0877 - acc:

In [56]:
predicted = model.predict(test_cnn_data, batch_size=1024, verbose=1)
columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
dfTestPredicted = pd.DataFrame(columns=columns)
for x in tqdm(range(len(test))):
    dfTestPredicted.loc[x] = [test['id'][x], predicted[x][1], predicted[x][2], predicted[x][3], predicted[x][4], predicted[x][5], predicted[x][6]]
dfTestPredicted.to_csv('../reports/testPred/predTestCNNW2C'+ str(topNFeatures) +'.csv',encoding='utf-8', index=False)

  3072/159571 [..............................] - ETA: 5: - ETA: 5: - ETA: 5:15

C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


159571/159571 [==============================] - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 5: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - E


  5%|▍         | 7004/153164 [00:15<05:25, 448.92it/s]


  7%|▋         | 11429/153164 [00:31<06:32, 361.08it/s]


...

 98%|█████████▊| 150017/153164 [42:33<00:53, 58.76it/s]


 99%|█████████▊| 150892/153164 [42:49<00:38, 58.71it/s]


 99%|█████████▉| 151767/153164 [43:06<00:23, 58.67it/s]


100%|█████████▉| 152643/153164 [43:24<00:08, 58.62it/s]


100%|██████████| 153164/153164 [43:34<00:00, 58.58it/s]


C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
